In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
base_url = "https://restproperty.com/filter/?type-search=44&rooms=&type-country=1&reginput[0]=47&price1=&price2=&objectid=&area-from=&area-to=&etag-from=&etag-to=&year-from=&year-to=&sea-from=&sea-to=&aero=&elcount=48&PAGEN_1="

all_links = []

for page in range(1, 53):
    url = base_url + str(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Предположим, что ссылки на квартиры имеют класс 'apartment-link', замените это на актуальный класс ссылок
    links = soup.find_all('a', class_='link-detail-apart')
    
    for link in links:
        href = link.get('href')
        if href:
            all_links.append(href)
    
    # Задержка на 1 секунду
    time.sleep(1)

# Сохраняем ссылки в CSV файл
df = pd.DataFrame(all_links, columns=['Link'])
df.to_csv('apartment_links.csv', index=False)

print("Ссылки сохранены в файл apartment_links.csv")

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# Функция для парсинга данных с каждой страницы квартиры
def parse_apartment_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    short_info = {}
    facility_infrastructure = []
    description = ''

    # Парсинг краткой информации
    short_info_section = soup.find('div', class_='detail-params-property-items')
    if short_info_section:
        for item in short_info_section.find_all('li', class_='detail-style-ln'):
            key_value = item.get_text(separator=": ").split(': ')
            if len(key_value) == 2:
                short_info[key_value[0].strip()] = key_value[1].strip()
    
    # Парсинг инфраструктуры
    facility_section = soup.find('ul', class_='infrastructure-list')
    if facility_section:
        for item in facility_section.find_all('li', class_='infrastructure-item'):
            facility_infrastructure.append(item.get_text(strip=True))
    
    # Парсинг описания
    description_section = soup.find('div', class_='new-detail-description-text')
    if description_section:
        description = description_section.get_text(strip=True)
    
    # Собираем все данные в один словарь
    data = short_info
    data['Facility infrastructure'] = ', '.join(facility_infrastructure)
    data['Description'] = description
    data['URL'] = url
    
    return data

# Загрузка ссылок из CSV файла
links_df = pd.read_csv('apartment_links.csv')
apartment_data = []

# Парсинг данных с каждой страницы квартиры
for link in tqdm(links_df['Link']):
    full_url = "https://restproperty.com" + link
    try:
        data = parse_apartment_page(full_url)
        apartment_data.append(data)
    except Exception as e:
        print(f"Error parsing {full_url}: {e}")

# Сохранение данных в XLS файл
apartment_df = pd.DataFrame(apartment_data)
apartment_df.to_excel('apartment_data.xlsx', index=False)

print("Данные сохранены в файл apartment_data.xlsx")


100%|██████████| 4/4 [00:04<00:00,  1.12s/it]

Данные сохранены в файл apartment_data.xlsx


,Property type,Rooms,Gross area,Floor,Furniture,Sea view,To the sea,To the center,Offer,Year of construction,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,Apartments,1+1,60 m²,2,yes,no,750 м,500 м,From the owner / From the owner,2014.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apartments,2+1,125 m²,7,yes,yes,50 м,100 м,From the owner / From the owner,2007.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apartments,2+1,110 m²,5,yes,yes,200 м,100 м,From the owner / From the owner,2008.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Villas,5+1,200 m²,NaN,yes,yes,6 км,5 км,From the owner / From the owner,2019.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Villas,4+2,220 m²,NaN,yes,no,1 км,700 м,From the owner / From the owner,2004.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,Villas,4+1,200 - 250 m²,NaN,no,no,600 м,400 м,From the developer / From the developer,2022.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2485,Apartments,2+1,100 m²,NaN,no,no,500 м,200 м,From the developer / From the developer,2024.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2486,Villas,3+1,210 m²,NaN,yes,yes,3 км,4 км,From the owner / From the owner,2007.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2487,Penthouse,"2+1, 3+1, 4+1",110 - 195 m²,NaN,no,no,200 м,100 м,From the developer / From the developer,2021.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df

,Property type,Rooms,Gross area,Floor,Furniture,Sea view,To the sea,To the center,Offer,Year of construction,...,Roman steam room,Satellite TV,Shuttle to the beach,Table tennis,Tennis court,Turkish hamam,Video surveillance 24/7,Volleyball Court,Walking paths,Wireless Internet
0,Apartments,1+1,60 m²,2,yes,no,750 м,500 м,From the owner / From the owner,2014.0,...,1,1,0,1,1,1,1,0,1,1
1,Apartments,2+1,125 m²,7,yes,yes,50 м,100 м,From the owner / From the owner,2007.0,...,0,1,0,0,0,0,0,0,0,1
2,Apartments,2+1,110 m²,5,yes,yes,200 м,100 м,From the owner / From the owner,2008.0,...,0,1,0,0,0,0,1,0,0,1
3,Villas,5+1,200 m²,NaN,yes,yes,6 км,5 км,From the owner / From the owner,2019.0,...,0,1,0,1,0,0,1,0,0,1
4,Villas,4+2,220 m²,NaN,yes,no,1 км,700 м,From the owner / From the owner,2004.0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,Villas,4+1,200 - 250 m²,NaN,no,no,600 м,400 м,From the developer / From the developer,2022.0,...,0,1,0,0,0,0,1,0,0,1
2485,Apartments,2+1,100 m²,NaN,no,no,500 м,200 м,From the developer / From the developer,2024.0,...,1,1,0,1,1,1,1,1,1,1
2486,Villas,3+1,210 m²,NaN,yes,yes,3 км,4 км,From the owner / From the owner,2007.0,...,0,1,0,0,0,0,1,0,0,1
2487,Penthouse,"2+1, 3+1, 4+1",110 - 195 m²,NaN,no,no,200 м,100 м,From the developer / From the developer,2021.0,...,1,1,0,1,1,1,1,1,0,1


KeyError: 'Gross area'

KeyError: 'Furniture'

   To the sea  To the center  To the airport
0      0.7500          0.500           0.065
1      0.0500          0.100           0.028
2      0.2000          0.100           0.026
3      0.0015          0.003           0.015
4      0.0020          0.005           0.050
